<a href="https://colab.research.google.com/github/sulochandhungel/ReferenceET/blob/master/Agrimet_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Function to get AgriMet data
#Create a dataframe with day of satellite image day
import pandas as pd
import numpy as np
import requests
from io import StringIO

def getAgriMetdata(StnID, styr, stmon, stday,
                   enyr=None, enmon=None, enday=None,
                   ):
    if enyr is None:
        enyr = styr
    if enmon is None:
        enmon = stmon
    if enday is None:
        enday = stday
    #"pn-bin/webdaycsv.pl?"+\ # This stopped working
    Dataurl = "https://www.usbr.gov/"+\
    "pn-bin/instant.pl?"+\
    "station="+ StnID +\
    "&year=" + str(styr) +\
    "&month=" + str(stmon) +\
    "&day="+ str(stday) +\
    "&year="+ str(enyr) +\
    "&month="+ str(enmon) +\
    "&day="+ str(enday) +\
    "&pcode=" + "OB" +\
    "&pcode=" + "TU" +\
    "&pcode=" + "WS" +\
    "&pcode=" + "WD" +\
    "&pcode=" + "SI" +\
    "&pcode=" + "EA" +\
    "&pcode=" + "PC" +\
    "&pcode=" + "TP" +\
    "&pcode=" + "SL" +\
    "&pcode=" + "EH"


    #url = "https://www.usbr.gov/pn-bin/instant.pl?list=boii%20ob,boii%20obx&start=2016-04-15&end=2016-04-20"
    headers = {"User-Agent":
               "Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
                AppleWebKit/537.36 (KHTML, like Gecko) \
                Chrome/80.0.3987.163 Safari/537.36"}
    req = requests.get(Dataurl, headers=headers)
    Datalink = StringIO(req.text)
    #print (Datalink)
    
    df = pd.read_csv(Datalink, skiprows= 20, error_bad_lines=False, delim_whitespace=False) # with the new Agrimet data table, this has been changed
    #print (df.head(2))
    df.drop(df.tail(3).index,inplace=True)

    return df
print ("Done!")
StnID = "HRHW"
styr = 2015
stmon = 5
stday = 5
tt = getAgriMetdata(StnID, styr, stmon, stday)

Done!


In [0]:
import datetime
DTs = ["2016-04-08 11:43:18", "2016-04-16 11:45:58", "2016-05-02 11:46:04",\
           "2016-05-10 11:43:14", "2016-05-18 11:46:05", "2016-06-19 11:46:14"]
loc_DTs = [datetime.datetime.strptime(loc_DT, "%Y-%m-%d %H:%M:%S") for loc_DT in DTs] 

print (loc_DTs)

[datetime.datetime(2016, 4, 8, 11, 43, 18), datetime.datetime(2016, 4, 16, 11, 45, 58), datetime.datetime(2016, 5, 2, 11, 46, 4), datetime.datetime(2016, 5, 10, 11, 43, 14), datetime.datetime(2016, 5, 18, 11, 46, 5), datetime.datetime(2016, 6, 19, 11, 46, 14)]


In [0]:

k = 0
j = 0
if k == 0:
  if j ==0:
    result = None
    for DT in loc_DTs:
        if result is None:
            result = getAgriMetdata(StnID, DT.year, DT.month, DT.day)
            #result['PC'][0] = 0.0
            #result['PC'] = result['PC'][1:96] - result['PC'][0:95] 
        else:
            result2 = getAgriMetdata(StnID, DT.year, DT.month, DT.day)
            #result2['PC'][0] = 0.0
            #result2['PC'] = result2['PC'][1:96] - result2['PC'][0:95] 
            result = pd.concat([result, result2])
    
    for i in range(0, len(result.keys())):
        if ('WS' in result.keys()[i]) and (StnID.upper() in result.keys()[i]):
            WS_varid = result.keys()[i]
        if 'WD' in result.keys()[i] and (StnID.upper() in result.keys()[i]):
            WD_varid = result.keys()[i]
        if (('SI' in result.keys()[i]) and (not 'SI2' in result.keys()[i]) and\
            (StnID.upper() in result.keys()[i])):
            SI_varid = result.keys()[i]
        if 'TU' in result.keys()[i] and (StnID.upper() in result.keys()[i]):
            TU_varid = result.keys()[i]
        if 'PC' in result.keys()[i] and (StnID.upper() in result.keys()[i]):
            PC_varid = result.keys()[i]
        if (('OB' in result.keys()[i]) and (StnID.upper() in result.keys()[i])):
            OB_varid = result.keys()[i]
    
    DT_strname_in_table = result.iloc[1].index[0]
    #print (DT_strname_in_table)
    DateTime = pd.to_datetime((result[DT_strname_in_table]), format = '%m/%d/%Y %H:%M').rename('TIMESTAMP')
    records = pd.DataFrame(range(1, (DateTime.size)+1), columns= ['RECORD'])
    #records = pd.Series([range(1, (DateTime.size+1))]).rename('RECORD').reset_index(drop=True)
    #hourly_data1 = pd.concat([result, DateTime], axis=1).reset_index(drop=True)
    #hour_data = pd.concat([hourly_data1, records], axis=1)
    
    #print (result.keys())

    years = DateTime.dt.year.rename('Year')
    months = DateTime.dt.month.rename('Month')
    days = DateTime.dt.day.rename('Day')
    hours = DateTime.dt.hour.rename('Hour')
    minutes = DateTime.dt.minute.rename('Minute')
    seconds = DateTime.dt.second.rename('Second')

    ws_ms_avg = result[WS_varid]
    winddir = result[WD_varid]
    slrkw_avg = result[SI_varid]
    airtc_avg = result[OB_varid]
    rh = result[TU_varid]
    rainmmtot = result[PC_varid]

    #print (slrkw_avg * 1.0)
    xx = slrkw_avg.apply(pd.to_numeric)
    
    #print ("All rows numeric?")
    #print (result.applymap(np.isreal).all(axis = 0))
    #print (type(rh))
    
    print ("Forcing to nan if not numeric...")
    rh = (rh.apply(pd.to_numeric, errors='coerce')).rename('RH')
    ws_ms_avg = ((ws_ms_avg.apply(pd.to_numeric, errors='coerce')) * 0.44704).rename('WS_ms_Avg')
    winddir = (winddir.apply(pd.to_numeric, errors='coerce')).rename('WindDir')
    slrkw_avg = (((slrkw_avg.apply(pd.to_numeric, errors='coerce'))) * (41868.0 / (1000.0 * 3600.0))).rename('SlrkW_Avg')
    airtc_avg = (((airtc_avg.apply(pd.to_numeric, errors='coerce'))-32.0) * (5/9.0)).rename('AirTC_Avg')
    rainmmtot = (rainmmtot.apply(pd.to_numeric, errors='coerce') * 25.4).rename('Rain_mm_Tot')
    
    list_of_series = [DateTime, years, months, days, hours, minutes, seconds,
                     ws_ms_avg, winddir, slrkw_avg, airtc_avg, rh, rainmmtot]
    hourly_data = None
    for ser in list_of_series:
        if hourly_data is None:
            hourly_data = pd.DataFrame(ser)
        else:
            hourly_data = pd.concat([hourly_data, ser], axis=1)
    hourly_data = hourly_data.reset_index(drop=True)
    hourly_data = pd.concat([hourly_data, records], axis=1)
    #print (hourly_data)

    # Create a complete dataframe for all the variables
    w_df = pd.DataFrame().reindex_like(hourly_data).iloc[0:len(loc_DTs)]
    w_df['TIMESTAMP'] = loc_DTs

    w_df1 = pd.concat([hourly_data, w_df]).sort_values(by=['TIMESTAMP']).reset_index(drop=True)
    indsNaNs =  (w_df1[w_df1['RECORD'].isnull()].index) # only those datetimes are interpolated for Satellite datetimes
    #indsNaNs =  (w_df1[w_df1.isnull().any(axis=1)].index) # all NaNs are interpolated 
    #print (indsNaNs)

    w_df1 = w_df1.replace(-9999.000000, np.nan)
    w_df_interp = w_df1.interpolate('linear')

    w_df_4_Sat_DT_agrimet = w_df_interp.iloc[indsNaNs]
    w_df_4_Sat_DT = w_df_4_Sat_DT_agrimet
    #st_date_for_daily = loc_DTs[0] - timedelta(days=60)
    #en_date_for_daily = loc_DTs[len(loc_DTs)-1]

w_df_4_Sat_DT

Forcing to nan if not numeric...


,TIMESTAMP,Year,Month,Day,Hour,Minute,Second,WS_ms_Avg,WindDir,SlrkW_Avg,AirTC_Avg,RH,Rain_mm_Tot,RECORD
47,2016-04-08 11:43:18,2016.0,4.0,8.0,11.0,37.5,0.0,1.401470,117.00,0.702917,23.136111,34.535,472.694,47.5
143,2016-04-16 11:45:58,2016.0,4.0,16.0,11.5,22.5,0.0,1.718869,122.90,0.738563,15.330556,50.390,476.504,142.5
238,2016-05-02 11:46:04,2016.0,5.0,2.0,11.5,22.5,0.0,1.832864,165.10,0.795608,23.111111,35.540,476.504,236.5
332,2016-05-10 11:43:14,2016.0,5.0,10.0,11.0,37.5,0.0,2.344725,158.50,0.881089,19.697222,31.565,0.000,329.5
428,2016-05-18 11:46:05,2016.0,5.0,18.0,11.5,22.5,0.0,1.955800,136.30,0.887543,25.408333,38.770,20.066,424.5
523,2016-06-19 11:46:14,2016.0,6.0,19.0,11.5,22.5,0.0,2.013915,96.22,0.939297,19.136111,29.380,21.336,518.5
